In [1]:
import dateutil

df_phone = pd.read_csv("code/ch5/data/phone_data.csv")
df_phone.tail()

,index,date,duration,item,month,network,network_type
825,825,13/03/15 00:38,1.000,sms,2015-03,world,world
826,826,13/03/15 00:39,1.000,sms,2015-03,Vodafone,mobile
827,827,13/03/15 06:58,34.429,data,2015-03,data,data
828,828,14/03/15 00:13,1.000,sms,2015-03,world,world
829,829,14/03/15 00:16,1.000,sms,2015-03,world,world


In [2]:
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.tail()

,index,date,duration,item,month,network,network_type
825,825,2015-03-13 00:38:00,1.000,sms,2015-03,world,world
826,826,2015-03-13 00:39:00,1.000,sms,2015-03,Vodafone,mobile
827,827,2015-03-13 06:58:00,34.429,data,2015-03,data,data
828,828,2015-03-14 00:13:00,1.000,sms,2015-03,world,world
829,829,2015-03-14 00:16:00,1.000,sms,2015-03,world,world


In [3]:
# 월별 사용량 합계
df_phone.groupby("month")['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [4]:
# item == call, network별 사용량 합계
df_phone[df_phone['item'] == 'call'].groupby("network")['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [5]:
# month/item별 count
df_phone.groupby(['month','item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

date로 count하던 network로 count하던 똑같다.

In [9]:
# unstack
df_phone.groupby(['month','item'])['date'].count().unstack()

item,call,data,sms
month,,,
2014-11,107,29,94
2014-12,79,30,48
2015-01,88,31,86
2015-02,67,31,39
2015-03,47,29,25


In [18]:
df_phone.groupby('month', as_index=False)['duration'].sum()

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


as_index는 'index를 따로 만들지 마라'라는 의미
- True면 안만들고, False면 만든다.

##### aggregation

In [19]:
df_phone.groupby('month', as_index=False).agg({"duration" : sum})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


위 결과와 같네...형태는 key : value형태

In [23]:
df_phone.groupby(['month','item']).agg({'duration':'sum',
                                       'network':'count',
                                       'date':'first'})

duration  network                date
month   item                                        
2014-11 call  25547.000      107 2014-10-15 06:58:00
        data    998.441       29 2014-10-15 06:58:00
        sms      94.000       94 2014-10-16 22:18:00
2014-12 call  13561.000       79 2014-11-14 17:24:00
        data   1032.870       30 2014-11-13 06:58:00
        sms      48.000       48 2014-11-14 17:28:00
2015-01 call  17070.000       88 2014-12-15 20:03:00
        data   1067.299       31 2014-12-13 06:58:00
        sms      86.000       86 2014-12-15 19:56:00
2015-02 call  14416.000       67 2015-01-15 10:36:00
        data   1067.299       31 2015-01-13 06:58:00
        sms      39.000       39 2015-01-15 12:23:00
2015-03 call  21727.000       47 2015-02-12 20:15:00
        data    998.441       29 2015-02-13 06:58:00
        sms      25.000       25 2015-02-19 18:46:00

##### 하나의 column에 여러개의 agg적용

In [27]:
df_phone.groupby(['month','item']).agg({'duration':['sum','max','min'],
                                       'network':'count',
                                       'date':['first','nunique']})

duration                    network                date        
                    sum        max     min   count               first nunique
month   item                                                                  
2014-11 call  25547.000   1940.000   1.000     107 2014-10-15 06:58:00     104
        data    998.441     34.429  34.429      29 2014-10-15 06:58:00      29
        sms      94.000      1.000   1.000      94 2014-10-16 22:18:00      79
2014-12 call  13561.000   2120.000   2.000      79 2014-11-14 17:24:00      76
        data   1032.870     34.429  34.429      30 2014-11-13 06:58:00      30
        sms      48.000      1.000   1.000      48 2014-11-14 17:28:00      41
2015-01 call  17070.000   1859.000   2.000      88 2014-12-15 20:03:00      84
        data   1067.299     34.429  34.429      31 2014-12-13 06:58:00      31
        sms      86.000      1.000   1.000      86 2014-12-15 19:56:00      58
2015-02 call  14416.000   1863.000   1.000      67 2015-01-15 10:36:00      67
        data   1067.299     34.429  34.429      31 2015-01-13 06:58:00      31
        sms      39.000      1.000   1.000      39 2015-01-15 12:23:00      27
2015-03 call  21727.000  10528.000   2.000      47 2015-02-12 20:15:00      47
        data    998.441     34.429  34.429      29 2015-02-13 06:58:00      29
        sms      25.000      1.000   1.000      25 2015-02-19 18:46:00      17

##### 위에 두 줄로 되어있는 column명을 1줄로 바꾸기 

In [32]:
grouped = df_phone.groupby(['month','item']).agg({'duration':['sum','max','min'],
                                       'network':'count',
                                       'date':['first','nunique']})

In [33]:
grouped.columns = grouped.columns.droplevel(level=0)
grouped

sum        max     min  count               first  nunique
month   item                                                                  
2014-11 call  25547.000   1940.000   1.000    107 2014-10-15 06:58:00      104
        data    998.441     34.429  34.429     29 2014-10-15 06:58:00       29
        sms      94.000      1.000   1.000     94 2014-10-16 22:18:00       79
2014-12 call  13561.000   2120.000   2.000     79 2014-11-14 17:24:00       76
        data   1032.870     34.429  34.429     30 2014-11-13 06:58:00       30
        sms      48.000      1.000   1.000     48 2014-11-14 17:28:00       41
2015-01 call  17070.000   1859.000   2.000     88 2014-12-15 20:03:00       84
        data   1067.299     34.429  34.429     31 2014-12-13 06:58:00       31
        sms      86.000      1.000   1.000     86 2014-12-15 19:56:00       58
2015-02 call  14416.000   1863.000   1.000     67 2015-01-15 10:36:00       67
        data   1067.299     34.429  34.429     31 2015-01-13 06:58:00       31
        sms      39.000      1.000   1.000     39 2015-01-15 12:23:00       27
2015-03 call  21727.000  10528.000   2.000     47 2015-02-12 20:15:00       47
        data    998.441     34.429  34.429     29 2015-02-13 06:58:00       29
        sms      25.000      1.000   1.000     25 2015-02-19 18:46:00       17

In [34]:
grouped.rename(columns={'sum' : 'sum_duration',
                        'max' : 'max_duration',
                       'min' : 'min_duration',
                       'count' : 'count_network',
                       'first' : 'first_date',
                       'nunique' : 'num_of_unique',})

sum_duration  max_duration  min_duration  count_network  \
month   item                                                            
2014-11 call     25547.000      1940.000         1.000            107   
        data       998.441        34.429        34.429             29   
        sms         94.000         1.000         1.000             94   
2014-12 call     13561.000      2120.000         2.000             79   
        data      1032.870        34.429        34.429             30   
        sms         48.000         1.000         1.000             48   
2015-01 call     17070.000      1859.000         2.000             88   
        data      1067.299        34.429        34.429             31   
        sms         86.000         1.000         1.000             86   
2015-02 call     14416.000      1863.000         1.000             67   
        data      1067.299        34.429        34.429             31   
        sms         39.000         1.000         1.000             39   
2015-03 call     21727.000     10528.000         2.000             47   
        data       998.441        34.429        34.429             29   
        sms         25.000         1.000         1.000             25   

                      first_date  num_of_unique  
month   item                                     
2014-11 call 2014-10-15 06:58:00            104  
        data 2014-10-15 06:58:00             29  
        sms  2014-10-16 22:18:00             79  
2014-12 call 2014-11-14 17:24:00             76  
        data 2014-11-13 06:58:00             30  
        sms  2014-11-14 17:28:00             41  
2015-01 call 2014-12-15 20:03:00             84  
        data 2014-12-13 06:58:00             31  
        sms  2014-12-15 19:56:00             58  
2015-02 call 2015-01-15 10:36:00             67  
        data 2015-01-13 06:58:00             31  
        sms  2015-01-15 12:23:00             27  
2015-03 call 2015-02-12 20:15:00             47  
        data 2015-02-13 06:58:00             29  
        sms  2015-02-19 18:46:00             17

In [35]:
grouped.add_prefix('prefix_')

prefix_sum  prefix_max  prefix_min  prefix_count  \
month   item                                                     
2014-11 call   25547.000    1940.000       1.000           107   
        data     998.441      34.429      34.429            29   
        sms       94.000       1.000       1.000            94   
2014-12 call   13561.000    2120.000       2.000            79   
        data    1032.870      34.429      34.429            30   
        sms       48.000       1.000       1.000            48   
2015-01 call   17070.000    1859.000       2.000            88   
        data    1067.299      34.429      34.429            31   
        sms       86.000       1.000       1.000            86   
2015-02 call   14416.000    1863.000       1.000            67   
        data    1067.299      34.429      34.429            31   
        sms       39.000       1.000       1.000            39   
2015-03 call   21727.000   10528.000       2.000            47   
        data     998.441      34.429      34.429            29   
        sms       25.000       1.000       1.000            25   

                    prefix_first  prefix_nunique  
month   item                                      
2014-11 call 2014-10-15 06:58:00             104  
        data 2014-10-15 06:58:00              29  
        sms  2014-10-16 22:18:00              79  
2014-12 call 2014-11-14 17:24:00              76  
        data 2014-11-13 06:58:00              30  
        sms  2014-11-14 17:28:00              41  
2015-01 call 2014-12-15 20:03:00              84  
        data 2014-12-13 06:58:00              31  
        sms  2014-12-15 19:56:00              58  
2015-02 call 2015-01-15 10:36:00              67  
        data 2015-01-13 06:58:00              31  
        sms  2015-01-15 12:23:00              27  
2015-03 call 2015-02-12 20:15:00              47  
        data 2015-02-13 06:58:00              29  
        sms  2015-02-19 18:46:00              17